In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
from utils.seed import fix_seed
import random
import os
from answer_judge.vqaEval import VQAEval

In [2]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/llava')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is the person wearing a hat?\nAnswer the ques...,"[{'answer': ['no'], 'answer_confidence': ['yes...",[tensor(393230010)],{'embedding': [[[ 0.00039101 -0.00074005 0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[What objects are directly above the TV set?\n...,"[{'answer': ['pictures'], 'answer_confidence':...",[tensor(131312003)],{'embedding': [[[-4.5394897e-04 9.3460083e-04...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many ski boards are in the picture?\nAnsw...,"[{'answer': ['5'], 'answer_confidence': ['yes'...",[tensor(201001)],{'embedding': [[[ 0.00125885 -0.00066376 0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[What color is the toothbrush?\nAnswer the que...,"[{'answer': ['white and purple'], 'answer_conf...",[tensor(393223001)],{'embedding': [[[ 0.00022602 -0.00056076 -0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[What is the green vegetable?\nAnswer the ques...,"[{'answer': ['i don't know'], 'answer_confiden...",[tensor(262552002)],{'embedding': [[[ 0.00182343 -0.00123596 0.00...,None


In [5]:
df.shape

(1519, 6)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000393230.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['No']

In [9]:
df.iloc[i]['question']

['Is the person wearing a hat?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([1])},
 {'answer': ['yes'], 'answer_confidence': ['maybe'], 'answer_id': tensor([2])},
 {'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([3])},
 {'answer': ['yes'], 'answer_confidence': ['yes'], 'answer_id': tensor([4])},
 {'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([5])},
 {'answer': ['yes'], 'answer_confidence': ['yes'], 'answer_id': tensor([6])},
 {'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([7])},
 {'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([8])},
 {'answer': ['no'], 'answer_confidence': ['yes'], 'answer_id': tensor([9])},
 {'answer': ['yes'],
  'answer_confidence': ['maybe'],
  'answer_id': tensor([10])}]

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(1519, 33, 4096)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
rsps[0]

'No'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/1519 [00:00<?, ?it/s]

100%|██████████| 1519/1519 [00:00<00:00, 3019.91it/s]


Done computing accuracy
{0: 100, 1: 100, 2: 33.33, 3: 100, 4: 66.67, 5: 100, 6: 100, 7: 100, 8: 33.33, 9: 100, 10: 100, 11: 100, 12: 100, 13: 100, 14: 100, 15: 100, 16: 100, 17: 100, 18: 100, 19: 100, 20: 100, 21: 100, 22: 0.0, 23: 100, 24: 100, 25: 100, 26: 100, 27: 100, 28: 100, 29: 100, 30: 100, 31: 100, 32: 0.0, 33: 100, 34: 100, 35: 100, 36: 100, 37: 100, 38: 100, 39: 100, 40: 0.0, 41: 100, 42: 100, 43: 100, 44: 100, 45: 100, 46: 0.0, 47: 100, 48: 100, 49: 100, 50: 100, 51: 100, 52: 100, 53: 100, 54: 100, 55: 66.67, 56: 0.0, 57: 100, 58: 100, 59: 100, 60: 0.0, 61: 100, 62: 100, 63: 100, 64: 0.0, 65: 100, 66: 100, 67: 100, 68: 100, 69: 100, 70: 100, 71: 100, 72: 100, 73: 100, 74: 100, 75: 0.0, 76: 100, 77: 66.67, 78: 100, 79: 0.0, 80: 100, 81: 33.33, 82: 100, 83: 100, 84: 0.0, 85: 100, 86: 0.0, 87: 100, 88: 100, 89: 100, 90: 0.0, 91: 0.0, 92: 100, 93: 100, 94: 100, 95: 100, 96: 0.0, 97: 100, 98: 33.33, 99: 33.33, 100: 0.0, 101: 100, 102: 33.33, 103: 100, 104: 100, 105: 100, 106: 0.

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(1519,)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 1519 responses. 1356 answer correctly and 163 answer wrongly


In [ ]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 200
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:515
validset length:700
testset length:304
trainset postive: 444 negative: 71
valset postive: 636 negative: 64
testset postive: 276 negative: 28


In [24]:
all_embeddings = embeddings
all_embeddings.shape

(1519, 33, 4096)

In [25]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [26]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [27]:
embedding_val.shape

(700, 33, 4096)

In [28]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(X[:,i_layer,:],n_components,M,method='CoP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [ ]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,11),
    'layer':range(0,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras,all_results=grid_search.search(return_results=True)

Grid Searching for best n_components,layer


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5678311713836477
found best n_components:1,layer:1 Score: 0.5811959512578616
found best n_components:1,layer:2 Score: 0.5969192216981132
found best n_components:1,layer:5 Score: 0.6076552672955975
found best n_components:1,layer:10 Score: 0.6163522012578616
found best n_components:1,layer:11 Score: 0.6166470125786163
found best n_components:1,layer:13 Score: 0.6205778301886793
found best n_components:1,layer:27 Score: 0.6264986242138365
found best n_components:1,layer:28 Score: 0.6265968946540881
found best n_components:1,layer:30 Score: 0.6422955974842767
found best n_components:1,layer:31 Score: 0.6472828223270441


  0%|          | 0/33 [00:00<?, ?it/s]

...

found best n_components:4,layer:22 Score: 0.6698358883647799


  0%|          | 0/33 [00:00<?, ?it/s]

found best n_components:5,layer:19 Score: 0.6839868317610062


  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [30]:
np.save('./VQA_layers.npy',all_results,allow_pickle=True)

In [ ]:
best_paras

{'n_components': 2, 'layer': 21, 'best_score': np.float64(0.7284065872512043)}

In [32]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [33]:
discriminator=KernelPCA(embedding_val[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [ ]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.6085714285714285 AUROC:0.6274572993876893


In [ ]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.5633802816901409 AUROC:0.6900380710659898


Train Linear Prob to Get Better Results

In [36]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [37]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [ ]:
clf.load_state_dict(torch.load('./pope_clf.pt', weights_only=True))

<All keys matched successfully>

In [ ]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.8309859154929577 f1:0.9072164948453608 auroc:0.6316624365482233  b-acc:0.4779505076142132 auc-pr:0.9600266946949687
